
setup the cds api key in PS

PS C:\Users\andre\Desktop\VS_Code\PMW_LST> @"
>> url:https://cds.climate.copernicus.eu/api/v2
>> key:304697:d5598eeb-1c86-43e5-b753-bd6fc93c562f   
>> "@| Set-Content -Path "$env:USERPROFILE\.cdsapiric" 



In [29]:
import cdsapi
import os
from datetime import timedelta, date

cds = cdsapi.Client()

### Test 1
Try to download a whole day, to later match each Windsat data point "time" with the closest value from ERA5-Land

In [25]:
dataset_name = "reanalysis-era5-land"

request = {
    "variable":[
        "skin_temperature",
        "soil_temperature_level_1",
        "temperature_of_snow_layer",
    ],
    "year": "2017",
    "month": "01",
    "day": "01",
    'time': [
        '00:00', '01:00', '02:00',
        '03:00', '04:00', '05:00',
        '06:00', '07:00', '08:00',
        '09:00', '10:00', '11:00',
        '12:00', '13:00', '14:00',
        '15:00', '16:00', '17:00',
        '18:00', '19:00', '20:00',
        '21:00', '22:00', '23:00',
    ],
    "format": "netcdf.zip",
}

target = "../data/raw/ERA5_land/Jan1st2017.netcdf.zip"

if  not os.path.exists(target):
    try:
        cds.retrieve(
            name=dataset_name,
            request=request,
            target=target,
        )
    except:
        #Delete the unfinished file:
        os.remove(target)
        print(f"Removed {target} file.")

else:
    print(f"Data file {target} allready exists, donwload skipped.")

2024-05-02 09:25:18,800 INFO Welcome to the CDS
2024-05-02 09:25:18,801 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
2024-05-02 09:25:18,973 INFO Downloading https://download-0017.copernicus-climate.eu/cache-compute-0017/cache/data0/adaptor.mars.internal-1714633524.9217231-12934-13-2a61af53-189a-4adb-9e0a-b39785811897.zip to ../data/raw/ERA5_land/Jan1st2017.netcdf.zip (247M)
2024-05-02 09:38:49,091 INFO Download rate 312.3K/s   


Next, a simple loop to download all hours from a range of dates

In [22]:
dataset_name = "reanalysis-era5-land"
start_date = date(2017,1,1)
end_date = date(2017,1,7)

# Create the request using the starting date
year = str(start_date.year).zfill(4)
month = str(start_date.month).zfill(2)
day = str(start_date.day).zfill(2)

all_hours = [f"{hour:02d}:00" for hour in range(24)]

request = {
    "variable":
    [
        "skin_temperature",
        "soil_temperature_level_1",
        "temperature_of_snow_layer",
    ],
    "year": year,
    "month": month,
    "day": day,
    "time": all_hours,
    "format": "netcdf.zip",
}

target = f"../data/raw/ERA5_land/{request['year']}_{request['month']}_{request['day']}.zip"

if  not os.path.exists(target):
    try:
        cds.retrieve(
            name=dataset_name,
            request=request,
            target=target,
        )
    except:
        os.remove(target)
        print(f"Removed compromised file {target}.")

else:
    print(f"Data file {target} allready exists, donwload skipped.")


Data file ../data/raw/ERA5_land/Jan1st2017.netcdf.zip allready exists, donwload skipped.


### -Use asyncio- INTENTIONALY NOT SUPORTED
We _CANT_ make a few requests in batches to speed the download (at least in the erc server)

In [ ]:

# async def concurrent_download(dates:list[datetime.date], save_dir:str):
#     tasks =[]

#     # Call the api
#     cds = cdsapi.Client()

#     for date in dates:

#     return



In [45]:
# Loop over a list of dates:
def get_base_request() -> dict:
    all_hours = [f"{hour:02d}:00" for hour in range(24)]

    base_request = {
        "variable":
        [
            "skin_temperature",
            "soil_temperature_level_1",
            "temperature_of_snow_layer",
        ],
        "year": None,
        "month": None,
        "day": None,
        "time": all_hours,
        "format": "netcdf.zip",
    }
    return base_request


def single_download(cds:cdsapi.Client, date:date, folder_path:str) -> None:
    request = get_base_request()

    # Update the  base request using the date
    request["year"] = str(date.year).zfill(4)
    request["month"] = str(date.month).zfill(2)
    request["day"] = str(date.day).zfill(2)

    # Name the target file acordingly:
    filename = f"{request['year']}_{request['month']}_{request['day']}.zip"
    target = os.path.join(folder_path, filename)


    # Process only unexisting, full downloads
    if  not os.path.exists(target):
        try:
            cds.retrieve(
                name=dataset_name,
                request=request,
                target=target,
            )
        except:
            os.remove(target)
            print(f"Removed compromised file {target}.")

    else:
        print(f"Data file {target} allready exists, donwload skipped.")

def download_many(dates:list[date], save_dir:str) -> None:

    cds = cdsapi.Client()

    for i,date in enumerate(dates):
        print(f"Downloading atempt {i}/{len(dates)}.")
        single_download(cds, date, save_dir)
        print("NEXT")

    print("DONE")



In [46]:
start_date = date(2017,1,1)
end_date = date(2017,1,2)

dates = [
        start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1)
    ]
save_dir = "../data/raw/ERA5_land/"
%time download_many(dates, save_dir)

Data file ../data/raw/ERA5_land/2017_01_01.zip allready exists, donwload skipped.
NEXT
Data file ../data/raw/ERA5_land/2017_01_02.zip allready exists, donwload skipped.
NEXT
DONE
CPU times: total: 0 ns
Wall time: 1e+03 µs
